In [2]:
%pip install opencv-python opencv-python-headless

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np

main_image_path = 'main.jpeg'
template_image_path = 'template.jpeg'

main_image = cv2.imread(main_image_path)
template = cv2.imread(template_image_path)

if main_image is None:
    raise ValueError(f"Could not load the main image from path: {main_image_path}")
if template is None:
    raise ValueError(f"Could not load the template image from path: {template_image_path}")

main_image_gray = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

orb = cv2.ORB_create()

keypoints1, descriptors1 = orb.detectAndCompute(main_image_gray, None)
keypoints2, descriptors2 = orb.detectAndCompute(template_gray, None)

if descriptors1 is None or descriptors2 is None:
    raise ValueError("Descriptors cannot be None")

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda x: x.distance)

print(f"Number of keypoints in main image: {len(keypoints1)}")
print(f"Number of keypoints in template image: {len(keypoints2)}")
print(f"Number of matches found: {len(matches)}")

valid_matches = []
for match in matches:
    if match.trainIdx < len(keypoints2) and match.queryIdx < len(keypoints1):
        valid_matches.append(match)
    else:
        print(f"Invalid match found: trainIdx={match.trainIdx}, queryIdx={match.queryIdx}")

if len(valid_matches) == 0:
    raise ValueError("No valid matches found.")

num_matches_to_draw = min(10, len(valid_matches))

if num_matches_to_draw == 0:
    raise ValueError("Not enough matches to draw.")

try:
    match_img = cv2.drawMatches(template, keypoints2, main_image, keypoints1, valid_matches[:num_matches_to_draw], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imshow(match_img)
except Exception as e:
    print(f"Error during drawing matches: {e}")

src_pts = np.float32([keypoints2[m.trainIdx].pt for m in valid_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints1[m.queryIdx].pt for m in valid_matches]).reshape(-1, 1, 2)

try:
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    if M is None or mask is None:
        raise ValueError("Homography could not be computed.")
    matchesMask = mask.ravel().tolist()
    print(f"Homography matrix:\n{M}")
except Exception as e:
    print(f"Error during computing homography: {e}")

h, w = template.shape[:2]

pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

try:
    dst = cv2.perspectiveTransform(pts, M)
    print(f"Transformed points: {dst}")
    if dst is not None and len(dst) == 4:
        main_image_with_box = cv2.polylines(main_image.copy(), [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow(main_image_with_box)
    else:
        print("Homography transformation failed.")
except Exception as e:
    print(f"Error during perspective transformation: {e}")


Number of keypoints in main image: 500
Number of keypoints in template image: 362
Number of matches found: 155
Error during drawing matches: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)

Homography matrix:
[[1.00309154e+00 6.49328248e-03 1.91585028e+02]
 [2.93833829e-03 1.00652174e+00 1.72462433e+02]
 [3.16459288e-06 1.42752623e-05 1.00000000e+00]]
Transformed points: [[[191.58502 172.46243]]

 [[192.43915 400.6454 ]]

 [[422.09552 401.02808]]

 [[421.98892 173.01233]]]
Error during perspective transformation: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)



In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

main_image_path = 'main_image.jpg'
template_image_path = 'template_image.jpg'

main_image = cv2.imread(main_image_path)
template_image = cv2.imread(template_image_path)

if main_image is None:
    raise ValueError(f"Could not load the main image from path: {main_image_path}")
if template_image is None:
    raise ValueError(f"Could not load the template image from path: {template_image_path}")

scale_percent = 50
width = int(main_image.shape[1] * scale_percent / 100)
height = int(main_image.shape[0] * scale_percent / 100)
dim = (width, height)

resized_main_image = cv2.resize(main_image, dim, interpolation=cv2.INTER_AREA)
resized_template_image = cv2.resize(template_image, (int(template_image.shape[1] * scale_percent / 100), int(template_image.shape[0] * scale_percent / 100)), interpolation=cv2.INTER_AREA)

main_gray = cv2.cvtColor(resized_main_image, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(resized_template_image, cv2.COLOR_BGR2GRAY)

method = cv2.TM_CCOEFF_NORMED
result = cv2.matchTemplate(main_image, template_image, method)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

top_left = max_loc
bottom_right = (top_left[0] + resized_template_image.shape[1], top_left[1] + resized_template_image.shape[0])
cv2.rectangle(resized_main_image, top_left, bottom_right, (0, 255, 0), 2)

def round_to_rgb(pixel):
    r, g, b = pixel
    if r > g and r > b:
        return [255, 0, 0]  # Red
    elif g > r and g > b:
        return [0, 255, 0]  # Green
    elif b > r and b > g:
        return [0, 0, 255]  # Blue
    else:
        if g >= r and g >= b:
            return [0, 255, 0]
        elif r >= g and r >= b:
            return [255, 0, 0]
        else:
            return [0, 0, 255]

rounded_pixels = np.apply_along_axis(round_to_rgb, 2, isolated_pixels).astype(np.uint8)

total_pixels = rounded_pixels.shape[0] * rounded_pixels.shape[1]
blue_pixels = np.sum(np.all(rounded_pixels == [255, 0, 0], axis=2))
red_pixels = np.sum(np.all(rounded_pixels == [0, 0, 255], axis=2))
green_pixels = np.sum(np.all(rounded_pixels == [0, 255, 0], axis=2))

blue_percentage = (blue_pixels / total_pixels) * 100 if total_pixels != 0 else 0
red_percentage = (red_pixels / total_pixels) * 100 if total_pixels != 0 else 0
green_percentage = (green_pixels / total_pixels) * 100 if total_pixels != 0 else 0

print(f"Total Pixels: {total_pixels:.2f}")
print(f"Blue Pixels: {blue_pixels:.2f}")
print(f"Red Pixels: {red_pixels:.2f}")
print(f"Green Pixels: {green_pixels:.2f}")

print(f"Percentage of blue content: {blue_percentage:.2f}%")
print(f"Percentage of red content: {red_percentage:.2f}%")
print(f"Percentage of green content: {green_percentage:.2f}%")

# Display the result
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(resized_main_image, cv2.COLOR_BGR2RGB))
plt.gca().add_patch(plt.Rectangle(top_left, bottom_right[0]-top_left[0], bottom_right[1]-top_left[1], edgecolor='red', facecolor='none', linewidth=2))
plt.title("Template Matching Result")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(rounded_pixels, cv2.COLOR_BGR2RGB))
plt.title("Isolated and Rounded Pixels")

plt.show()

ValueError: Could not load the main image from path: main_image.jpg

In [7]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the main image and template image
main_image_path = 'main2.jpeg'  # Use the uploaded image as the main image
template_image_path = 'template.jpeg'  # Replace with the actual template path

def get_box(main_image_path, template_image_path):
    main_image = cv2.imread(main_image_path)
    template_image = cv2.imread(template_image_path)

    if main_image is None:
        raise ValueError(f"Could not load the main image from path: {main_image_path}")
    if template_image is None:
        raise ValueError(f"Could not load the template image from path: {template_image_path}")

    # Perform template matching
    method = cv2.TM_CCOEFF_NORMED
    result = cv2.matchTemplate(main_image, template_image, method)
    print("LOLZ", result)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    print("LMAOZ", min_val, max_val, max_loc)

    # Define the bounding box around the matched region
    top_left = max_loc
    bottom_right = (top_left[0] + template_image.shape[1], top_left[1] + template_image.shape[0])



    # Function to round pixel values to the closest primary color
    def round_to_rgb(pixel):
        r, g, b = pixel
        if r > g and r > b:
            return [255, 0, 0]  # Red
        elif g > r and g > b:
            return [0, 255, 0]  # Green
        elif b > r and b > g:
            return [0, 0, 255]  # Blue
        else:
            # For cases where colors are equal or very close, prioritize green > red > blue
            if g >= r and g >= b:
                return [0, 255, 0]
            elif r >= g and r >= b:
                return [255, 0, 0]
            else:
                return [0, 0, 255]

    
    return main_image, top_left, bottom_right[0] - top_left[0],  bottom_right[1] - top_left[1]



main_image, t1, t2, t3 = get_box(main_image_path, template_image_path)

# Apply the rounding function to the isolated pixels
rounded_pixels = np.apply_along_axis(round_to_rgb, 2, main_image).astype(np.uint8)

# Calculate the percentage of blue, red, and green content
total_pixels = rounded_pixels.shape[0] * rounded_pixels.shape[1]
blue_pixels = np.sum(np.all(rounded_pixels == [255, 0, 0], axis=2))
red_pixels = np.sum(np.all(rounded_pixels == [0, 0, 255], axis=2))
green_pixels = np.sum(np.all(rounded_pixels == [0, 255, 0], axis=2))

blue_percentage = (blue_pixels / total_pixels) * 100 if total_pixels != 0 else 0
red_percentage = (red_pixels / total_pixels) * 100 if total_pixels != 0 else 0
green_percentage = (green_pixels / total_pixels) * 100 if total_pixels != 0 else 0

# Print the percentage of each color for debugging
print(f"Total Pixels: {total_pixels:.2f}")
print(f"Blue Pixels: {blue_pixels:.2f}")
print(f"Red Pixels: {red_pixels:.2f}")
print(f"Green Pixels: {green_pixels:.2f}")

print(f"Percentage of blue content: {blue_percentage:.2f}%")
print(f"Percentage of red content: {red_percentage:.2f}%")
print(f"Percentage of green content: {green_percentage:.2f}%")

# Display the result
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(main_image, cv2.COLOR_BGR2RGB))
plt.gca().add_patch(plt.Rectangle(top_left, bottom_right[0] - top_left[0], bottom_right[1] - top_left[1], edgecolor='red', facecolor='none', linewidth=2))
plt.title("Template Matching Result")



plt.show()


LOLZ [[ 0.05323996  0.05454271  0.05600282 ... -0.06487519 -0.06611527
  -0.06756429]
 [ 0.05492735  0.05633271  0.05774741 ... -0.06570893 -0.06686088
  -0.06820977]
 [ 0.05656643  0.05821118  0.05978657 ... -0.06597841 -0.06720684
  -0.06860124]
 ...
 [-0.0507264  -0.04847419 -0.04566492 ... -0.02362148 -0.0229804
  -0.02217088]
 [-0.0473063  -0.04505855 -0.04236206 ... -0.02430872 -0.02338894
  -0.0223693 ]
 [-0.04409054 -0.04180269 -0.03935144 ... -0.02455087 -0.02357016
  -0.02252704]]
LMAOZ -0.3116562068462372 0.36593496799468994 (262, 281)
Total Pixels: 800000.00
Blue Pixels: 0.00
Red Pixels: 286186.00
Green Pixels: 513814.00
Percentage of blue content: 0.00%
Percentage of red content: 35.77%
Percentage of green content: 64.23%


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

main_image_path = 'main_image.jpg'
template_image_path = '/content/61KTbjfuNIL-removebg-preview.png'

main_image = cv2.imread(main_image_path)
template_image = cv2.imread(template_image_path)

if main_image is None:
    raise ValueError(f"Could not load the main image from path: {main_image_path}")
if template_image is None:
    raise ValueError(f"Could not load the template image from path: {template_image_path}")

scale_percent = 50
width = int(main_image.shape[1] * scale_percent / 100)
height = int(main_image.shape[0] * scale_percent / 100)
dim = (width, height)

resized_main_image = cv2.resize(main_image, dim, interpolation=cv2.INTER_AREA)
resized_template_image = cv2.resize(template_image, (int(template_image.shape[1] * scale_percent / 100), int(template_image.shape[0] * scale_percent / 100)), interpolation=cv2.INTER_AREA)

main_gray = cv2.cvtColor(resized_main_image, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(resized_template_image, cv2.COLOR_BGR2GRAY)

method = cv2.TM_CCOEFF_NORMED
result = cv2.matchTemplate(main_gray, template_gray, method)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

top_left = max_loc
bottom_right = (top_left[0] + resized_template_image.shape[1], top_left[1] + resized_template_image.shape[0])
cv2.rectangle(resized_main_image, top_left, bottom_right, (0, 255, 0), 2)

isolated_pixels = resized_main_image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

if isolated_pixels.size == 0:
    raise ValueError("Isolated pixels region is empty. Check the bounding box coordinates.")

def round_to_rgb(pixel):
    r, g, b = pixel
    if r > g and r > b:
        return [255, 0, 0]  # Red
    elif g > r and g > b:
        return [0, 255, 0]  # Green
    elif b > r and b > g:
        return [0, 0, 255]  # Blue
    else:
        if g >= r and g >= b:
            return [0, 255, 0]
        elif r >= g and r >= b:
            return [255, 0, 0]
        else:
            return [0, 0, 255]

rounded_pixels = np.apply_along_axis(round_to_rgb, 2, isolated_pixels).astype(np.uint8)

total_pixels = rounded_pixels.shape[0] * rounded_pixels.shape[1]
blue_pixels = np.sum(np.all(rounded_pixels == [255, 0, 0], axis=2))
red_pixels = np.sum(np.all(rounded_pixels == [0, 0, 255], axis=2))
green_pixels = np.sum(np.all(rounded_pixels == [0, 255, 0], axis=2))

blue_percentage = (blue_pixels / total_pixels) * 100 if total_pixels != 0 else 0
red_percentage = (red_pixels / total_pixels) * 100 if total_pixels != 0 else 0
green_percentage = (green_pixels / total_pixels) * 100 if total_pixels != 0 else 0

print(f"Total Pixels: {total_pixels:.2f}")
print(f"Blue Pixels: {blue_pixels:.2f}")
print(f"Red Pixels: {red_pixels:.2f}")
print(f"Green Pixels: {green_pixels:.2f}")

print(f"Percentage of blue content: {blue_percentage:.2f}%")
print(f"Percentage of red content: {red_percentage:.2f}%")
print(f"Percentage of green content: {green_percentage:.2f}%")

# Display the result
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(resized_main_image, cv2.COLOR_BGR2RGB))
plt.gca().add_patch(plt.Rectangle(top_left, bottom_right[0]-top_left[0], bottom_right[1]-top_left[1], edgecolor='red', facecolor='none', linewidth=2))
plt.title("Template Matching Result")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(rounded_pixels, cv2.COLOR_BGR2RGB))
plt.title("Isolated and Rounded Pixels")

plt.show()

Total Pixels: 62217.00
Blue Pixels: 19698.00
Red Pixels: 20722.00
Green Pixels: 21797.00
Percentage of blue content: 31.66%
Percentage of red content: 33.31%
Percentage of green content: 35.03%
